# 这个文件是下载并导入的文件

In [ ]:
import GEOparse
import datetime

In [ ]:
GEONAME = 'GSE63990'
# GSE63990  Profiling of bacterial respiratory infection, viral respiratory infection, and non-infectious illness
# GSE6269   Gene expression patterns in blood leukocytes discriminate patients with acute infections


In [ ]:
gse = GEOparse.get_GEO(geo=GEONAME, destdir="./datasets", silent=True)

In [ ]:
gse.get_metadata_attribute("title")

In [ ]:
# 先把一个个样本的Value值提取出来，然后再转置，这样就可以得到一个样本对应的所有基因的表达值。
sample_value = gse.pivot_samples('VALUE').T
sample_value.tail()

In [ ]:
value_desc = []
Value_Types = [
    'MAS5-calculated Signal intensity',
    'RMA normalized and Log2 transformed'
]
infection = []
n = 0
# value = [] we want 2-d dataframe
import pandas as pd
value = pd.DataFrame()
for i in gse.gsms:
    n += 1


    # 感染状态的处理
    characteristics_ch1 = gse.gsms[i].metadata['characteristics_ch1']   # the infection_status is in the characteristics_ch1
    infection_status = characteristics_ch1[-1].split(':')[1].strip()            # 但characteristics_ch1有的有两个值，有的有三个，取最后一个
    infection.append(infection_status)
    
    # 对数等的处理
    value_type = value_desc.append(gse.gsms[i].columns["description"][1])
    if gse.gsms[i].columns["description"][1] == 'RMA normalized and Log2 transformed':
        # 将sample_value 里第 n 行的每一个值转化为2的x次方
        value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
        # same line as above
    else:
        print(gse.gsms[i].columns["description"][1])
        print('Now just use the raw value')
        print('Samples: ', sample_value.iloc[n-1].head())
        print('-'*50)
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)

    # Value中的

value = value.T

# merge the infection status and value
value.insert(0, 'infection_status', infection)

value.head()

In [ ]:
# Save the data and Show the first 5 rows
value.to_csv('./datasets/' + GEONAME + '.csv', index=False)
value.head()

## 不用了

不清楚什么时候再把这个id转化成通用的Gene Symbol。

可能在这里就转，也可以分析结束了再转。

大概需要10分钟

In [ ]:
# GPL571 = gse.gpls['GPL571']
# n = 0
# for rows in sample_value:
#     Gene_Symbol = GPL571.table[GPL571.table['ID'] == rows]['Gene Symbol']
#     if Gene_Symbol.empty:
#         Gene_Symbol[0] = rows
#     sample_value.rename(columns={rows: Gene_Symbol.values[0]}, inplace=True)
#     n += 1
#     if n % 100 == 0:
#         print("Finished %d rows of %d at %s" % (n, sample_value.shape[1], datetime.datetime.now()))
# sample_value.tail()

In [ ]:
# gse.gpls['GPL571'].table.columns # GPL571, 也就是这个GEO数据集的平台，的列。
# # 从这个平台可以直接读取Gene Symbol，Target Description等等有用的东西。

In [ ]:
# gse.gpls['GPL571'].table.head() # 前5行